# Tools and Dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Rescaling, BatchNormalization
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from google.colab import files
import os, zipfile
import numpy as np
from sklearn.metrics import (confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score)

# Getting the Dataset

In [2]:
!git clone --no-checkout https://github.com/Agasthya-Samyak-Jnan/Diabetic-Neuropathy-Risk-ML.git
%cd Diabetic-Neuropathy-Risk-ML
!git sparse-checkout init --cone
!git sparse-checkout set Datasets/Extended_Dataset
!git checkout main

Cloning into 'Diabetic-Neuropathy-Risk-ML'...
remote: Enumerating objects: 27269, done.
remote: Total 27269 (delta 0), reused 0 (delta 0), pack-reused 27269 (from 1)
Receiving objects: 100% (27269/27269), 1.14 GiB | 16.28 MiB/s, done.
Resolving deltas: 100% (1006/1006), done.
/content/Diabetic-Neuropathy-Risk-ML
Updating files: 100% (27990/27990), done.
Already on 'main'
Your branch is up to date with 'origin/main'.


# Global Parameters

Config Parameters

In [3]:
IMG_SIZE = 224
MODEL_NAME = "MobileNetV2"
VERSION = "Version-02"
train_dir = "Datasets/Extended_Dataset/train"
val_dir   = "Datasets/Extended_Dataset/val"

Hyperparameters

In [4]:
BATCH_SIZE = 128
LR = 1e-4
FT_LR = 1e-5
EPOCHS = 15
FT_EPOCHS = 5
tf.random.set_seed(42) # To reproduce same randomization
np.random.seed(42)  # To reproduce same randomization

# Prepare Dataset


In [5]:
# Define data generator object to load dataset samples during training
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="binary"
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)


Found 21660 files belonging to 2 classes.
Found 6330 files belonging to 2 classes.


# Model Architecture

In [6]:
# Define Model

# Input layer
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Normalize inside the model
x = Rescaling(1./255)(inputs)

# Pre-trained MobileNetV2 base
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_tensor=x  # pass normalized inputs here
)

base_model.trainable = False  # Phase 1: feature extraction

# Add classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

# Define final model
model = Model(inputs=inputs, outputs=output)

/tmp/ipython-input-3811952941.py:10: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


# Training the Model

Model Initilaization or Pre-loading (if pre-trained)

In [7]:
# Instantiate the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

Training - Phase 1 (First Training)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/15
170/170 ━━━━━━━━━━━━━━━━━━━━ 88s 377ms/step - accuracy: 0.5621 - auc: 0.5913 - loss: 0.8247 - val_accuracy: 0.8556 - val_auc: 0.9351 - val_loss: 0.3625
Epoch 2/15
170/170 ━━━━━━━━━━━━━━━━━━━━ 34s 201ms/step - accuracy: 0.7990 - auc: 0.8777 - loss: 0.4388 - val_accuracy: 0.8844 - val_auc: 0.9587 - val_loss: 0.2746
Epoch 3/15
170/170 ━━━━━━━━━━━━━━━━━━━━ 42s 206ms/step - accuracy: 0.8546 - auc: 0.9288 - loss: 0.3385 - val_accuracy: 0.8946 - val_auc: 0.9645 - val_loss: 0.2459
Epoch 4/15
170/170 ━━━━━━━━━━━━━━━━━━━━ 34s 200ms/step - accuracy: 0.8831 - auc: 0.9519 - loss: 0.2809 - val_accuracy: 0.8957 - val_auc: 0.9664 - val_loss: 0.2348
Epoch 5/15
170/170 ━━━━━━━━━━━━━━━━━━━━ 42s 206ms/step - accuracy: 0.8986 - auc: 0.9644 - loss: 0.2452 - val_accuracy: 0.8984 - val_auc: 0.9677 - val_loss: 0.2274
Epoch 6/15
 27/170 ━━━━━━━━━━━━━━━━━━━━ 26s 183ms/step - accuracy: 0.9086 - auc: 0.9682 - loss: 0.2318

In [ ]:
# Visualise training results
epochs = range(1, len(history.history['loss']) + 1)

# Accuracy
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training & Validation Performance')
plt.legend()
plt.grid(True)
plt.show()

# Loss
plt.plot(epochs, history.history['loss'], label='Train Loss')
plt.plot(epochs, history.history['val_loss'], label='Val Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Performance')
plt.legend()
plt.grid(True)
plt.show()

# AUC
plt.plot(epochs, history.history['auc'], label='Train AUC')
plt.plot(epochs, history.history['val_auc'], label='Val AUC')

plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.title('Training & Validation Performance')
plt.legend()
plt.grid(True)
plt.show()


Training - Phase 2 (Fine Tuning)

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FT_LR),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=FT_EPOCHS
)

In [ ]:
#  Visualise training results
epochs = range(1, len(history_finetune.history['loss']) + 1)

# Accuracy
plt.plot(epochs, history_finetune.history['accuracy'], label='Train Accuracy')
plt.plot(epochs, history_finetune.history['val_accuracy'], label='Val Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training & Validation Performance')
plt.legend()
plt.grid(True)
plt.show()

# Loss
plt.plot(epochs, history_finetune.history['loss'], label='Train Loss')
plt.plot(epochs, history_finetune.history['val_loss'], label='Val Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Performance')
plt.legend()
plt.grid(True)
plt.show()

# AUC
plt.plot(epochs, history_finetune.history['auc'], label='Train AUC')
plt.plot(epochs, history_finetune.history['val_auc'], label='Val AUC')

plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.title('Training & Validation Performance')
plt.legend()
plt.grid(True)
plt.show()

# Evaluation of Model

In [ ]:
# Compute final evaluation metrics

y_true = []
y_pred = []
y_prob = []

for images, labels in val_ds:
    probs = model.predict(images, verbose=0)
    preds = (probs > 0.5).astype(int)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)
    y_prob.extend(probs)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Metrics
accuracy  = (tp + tn) / (tp + tn + fp + fn)
precision = precision_score(y_true, y_pred)
recall    = recall_score(y_true, y_pred)
f1        = f1_score(y_true, y_pred)
auc       = roc_auc_score(y_true, y_prob)
npv       = tn / (tn + fn)

# Final validation loss (last epoch of fine-tuning)
final_loss = history_finetune.history["val_loss"][-1]

print(f"""
MODEL PERFORMANCE (Validation Set)

Accuracy  : {accuracy:.5f}
Precision : {precision:.5f}
Recall    : {recall:.5f}
F1-Score  : {f1:.5f}
AUC       : {auc:.5f}
NPV       : {npv:.5f}
Loss      : {final_loss:.5f}
""")

# Documentation and Saving the Model

In [ ]:
# Version and base paths
VERSION_NUMBER = VERSION[-2:]

BASE_PATH = f"{VERSION}"
STATS_DIR = os.path.join(BASE_PATH, "Training Stats")

os.makedirs(STATS_DIR, exist_ok=True)

# Save model (correct location + naming)
model.save(f"{MODEL_NAME}-base-model-v{VERSION_NUMBER}.keras")
MODEL_PATH = os.path.join(BASE_PATH,f"{MODEL_NAME}-base-model-v{VERSION_NUMBER}.keras")
model.save(MODEL_PATH)

# Save hyperparameters
hyperparam_text = f"""
Dataset Size = 27,990 Images
Training Size = 21,660 Images (~77.4%)
Validation Size = 6,330 Images (~22.6%)

Input Image Size = {IMG_SIZE}x{IMG_SIZE}
Batch Size = {BATCH_SIZE}

Loss Function = Binary Crossentropy (BCE)
Optimizer = Adam
Learning Rate = {LR}
Fine Tuning Learning Rate = {FT_LR}

1st Training Epochs = {EPOCHS}
Fine Tuning Epochs = {FT_EPOCHS}
"""

with open(os.path.join(BASE_PATH, f"Hyperparameters-v{VERSION_NUMBER}.txt"), "w") as f:
    f.write(hyperparam_text.strip())

# Save logs
def save_log(history, filename):
    with open(os.path.join(STATS_DIR, filename), "w") as f:
        for k, v in history.history.items():
            f.write(f"{k}: {v}\n")

save_log(history, "first_training_log.txt")
save_log(history_finetune, "fine_tuning_log.txt")

# Save plots
import matplotlib.pyplot as plt

def save_plot(history, metric, filename):
    plt.figure()
    plt.plot(history.history[metric], label=f"Train {metric}")
    plt.plot(history.history[f"val_{metric}"], label=f"Val {metric}")
    plt.xlabel("Epochs")
    plt.ylabel(metric.upper())
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(STATS_DIR, filename))
    plt.close()

# Phase 1
save_plot(history, "accuracy", "1stTraining_Accuracy.png")
save_plot(history, "loss", "1stTraining_Loss.png")
save_plot(history, "auc", "1stTraining_AUC.png")

# Phase 2
save_plot(history_finetune, "accuracy", "FineTuning_Accuracy.png")
save_plot(history_finetune, "loss", "FineTuning_Loss.png")
save_plot(history_finetune, "auc", "FineTuning_AUC.png")

# Save final evaluation metrics
performance_text = f"""
MODEL PERFORMANCE (Validation Set)

Accuracy  : {accuracy:.5f}
Precision : {precision:.5f}
Recall    : {recall:.5f}
F1-Score  : {f1:.5f}
AUC       : {auc:.5f}
NPV       : {npv:.5f}
Loss      : {final_loss:.5f}
"""

with open(os.path.join(BASE_PATH, "model_performance.txt"), "w") as f:
    f.write(performance_text.strip())

# Put everything into a Zip Folder
ZIP_NAME = f"{VERSION}.zip"

with zipfile.ZipFile(ZIP_NAME, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, file_list in os.walk(BASE_PATH):
        for file in file_list:
            full_path = os.path.join(root, file)
            arcname = os.path.relpath(full_path, BASE_PATH)
            zipf.write(full_path, arcname)

files.download(ZIP_NAME)